In [ ]:
!cat requirements.txt

In [ ]:
import sys
!{sys.executable} -m pip install -r requirements.txt

In [ ]:
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-python-adapter-hpds.git
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-python-client.git

In [1]:
PICSURE_network_URL = "https://biodatacatalyst.integration.hms.harvard.edu/picsure"
resource_id = "02e23f52-f354-4e8b-992c-d37c8b9ba140"
token_file = "token.txt"

In [2]:
with open(token_file, "r") as f:
    my_token = f.read()

In [3]:
import json
from pprint import pprint

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from scipy import stats

import PicSureHpdsLib
import PicSureClient

from python_lib.utils import get_multiIndex_variablesDict, joining_variablesDict_onCol

In [4]:
client = PicSureClient.Client()
connection = client.connect(PICSURE_network_URL, my_token, True)
adapter = PicSureHpdsLib.Adapter(connection)
resource = adapter.useResource(resource_id)



+=========================================================================================+
|        [ WARNING ] you are specifying that you WANT to allow self-signed SSL            |
|        certificates to be acceptable for connections.  This may be useful for           |
|        working in a development environment or on systems that host public              |
|        data.  BEST SECURITY PRACTICES ARE THAT IF YOU ARE WORKING WITH SENSITIVE        |
|        DATA THEN ALL SSL CERTS BY THOSE EVIRONMENTS SHOULD NOT BE SELF-SIGNED.          |
+=========================================================================================+

+--------------------------------------+------------------------------------------------------
|  Resource UUID                       |  Resource Name                                  
+--------------------------------------+------------------------------------------------------
| 02e23f52-f354-4e8b-992c-d37c8b9ba140
+---------------------------------

In [5]:
dictionary = resource.dictionary()
dictionary_search = dictionary.find("\\_consents\\").DataFrame()
consentGroups = dictionary_search["categoryValues"]["\\_consents\\"]
consentGroups

['phs000007.c0',
 'phs000007.c1',
 'phs000007.c2',
 'phs000179.c0',
 'phs000179.c1',
 'phs000179.c2',
 'phs000200.c0',
 'phs000200.c1',
 'phs000200.c2',
 'phs000209.c0',
 'phs000209.c1',
 'phs000209.c2',
 'phs000280.c0',
 'phs000280.c1',
 'phs000280.c2',
 'phs000284.c0',
 'phs000284.c1',
 'phs000286.c0',
 'phs000286.c1',
 'phs000286.c2',
 'phs000286.c3',
 'phs000286.c4',
 'phs000287.c0',
 'phs000287.c1',
 'phs000287.c2',
 'phs000287.c3',
 'phs000287.c4',
 'phs000422.c1',
 'phs000784.c0',
 'phs000784.c1',
 'phs000820.c1',
 'phs000920.c2',
 'phs000921.c2',
 'phs000946.c1',
 'phs000951.c1',
 'phs000951.c2',
 'phs000954.c1',
 'phs000956.c0',
 'phs000956.c2',
 'phs000964.c0',
 'phs000964.c1',
 'phs000964.c2',
 'phs000964.c3',
 'phs000964.c4',
 'phs000974.c1',
 'phs000974.c2',
 'phs000988.c0',
 'phs000988.c1',
 'phs000993.c1',
 'phs000993.c2',
 'phs000997.c1',
 'phs001001.c1',
 'phs001001.c2',
 'phs001013.c1',
 'phs001013.c2',
 'phs001024.c1',
 'phs001032.c1',
 'phs001040.c1',
 'phs001062.c1

In [7]:
import time

my_query = resource.query()
my_query.filter().delete("\\_topmed_consents\\")
my_query.filter().delete("\\_harmonized_consent\\")
dictionary = resource.dictionary()
output = []
batchSize = 1000
badConcepts = []
for consentGroup in consentGroups:
    if ".c0" not in consentGroup:
        print(consentGroup)
        consentGroupConcepts = dictionary.find(consentGroup.split(".")[0]).DataFrame()
        conceptsWithCounts = 0
        for i in range(0, len(consentGroupConcepts.index.values), batchSize):  
            my_query.crosscounts().clear()
            my_query.crosscounts().add(consentGroupConcepts.index.values[i:i + batchSize])
            my_query.filter().delete("\\_consents\\")
            my_query.filter().add("\\_consents\\", consentGroup)
            try:                
                cross_counts = my_query.getCrossCounts()
                cross_counts_list = [(k, v) for k, v in cross_counts.items() if v > 0] 
                cross_counts_empty = [(k, v) for k, v in cross_counts.items() if v == 0] 
                print(i)
                print(len(cross_counts_list))
                conceptsWithCounts += len(cross_counts_list)
            except:
                print("Retrying")
                time.sleep(5)
                badConcepts.extend(consentGroupConcepts.index.values[i:i + batchSize])
                cross_counts = my_query.getCrossCounts()
                cross_counts_list = [(k, v) for k, v in cross_counts.items() if v > 0] 
                cross_counts_empty = [(k, v) for k, v in cross_counts.items() if v == 0] 
                print("RETRY SUCCESS")
                print(i)
                print(len(cross_counts_list))
                conceptsWithCounts += len(cross_counts_list)
                pass
        output.append(consentGroup + "," + str(conceptsWithCounts))

Deleted key: \_topmed_consents\
Deleted key: \_harmonized_consent\
phs000007.c1
cleared list
Deleted key: \_consents\
0
928
cleared list
Deleted key: \_consents\
1000
911
cleared list
Deleted key: \_consents\
2000
921
cleared list
Deleted key: \_consents\
3000
912
cleared list
Deleted key: \_consents\
4000
914
cleared list
Deleted key: \_consents\
5000
910
cleared list
Deleted key: \_consents\
6000
914
cleared list
Deleted key: \_consents\
7000
908
cleared list
Deleted key: \_consents\
8000
917
cleared list
Deleted key: \_consents\
9000
916
cleared list
Deleted key: \_consents\
10000
913
cleared list
Deleted key: \_consents\
11000
896
cleared list
Deleted key: \_consents\
12000
918
cleared list
Deleted key: \_consents\
13000
901
cleared list
Deleted key: \_consents\
14000
912
cleared list
Deleted key: \_consents\
15000
898
cleared list
Deleted key: \_consents\
16000
917
cleared list
Deleted key: \_consents\
17000
895
cleared list
Deleted key: \_consents\
18000
921
cleared list
Deleted 

14000
956
cleared list
Deleted key: \_consents\
15000
963
cleared list
Deleted key: \_consents\
16000
952
cleared list
Deleted key: \_consents\
17000
476
phs000209.c2
cleared list
Deleted key: \_consents\
0
880
cleared list
Deleted key: \_consents\
1000
882
cleared list
Deleted key: \_consents\
2000
889
cleared list
Deleted key: \_consents\
3000
888
cleared list
Deleted key: \_consents\
4000
889
cleared list
Deleted key: \_consents\
5000
889
cleared list
Deleted key: \_consents\
6000
880
cleared list
Deleted key: \_consents\
7000
905
cleared list
Deleted key: \_consents\
8000
885
cleared list
Deleted key: \_consents\
9000
888
cleared list
Deleted key: \_consents\
10000
880
cleared list
Deleted key: \_consents\
11000
879
cleared list
Deleted key: \_consents\
12000
880
cleared list
Deleted key: \_consents\
13000
887
cleared list
Deleted key: \_consents\
14000
896
cleared list
Deleted key: \_consents\
15000
885
cleared list
Deleted key: \_consents\
16000
879
cleared list
Deleted key: \_co

In [8]:
for line in output:
    print(line)

phs000007.c1,54912
phs000007.c2,54206
phs000179.c1,336
phs000179.c2,324
phs000200.c1,5779
phs000200.c2,5420
phs000209.c1,16824
phs000209.c2,15494
phs000280.c1,13533
phs000280.c2,9625
phs000284.c1,2288
phs000286.c1,3247
phs000286.c2,3169
phs000286.c3,3354
phs000286.c4,3176
phs000287.c1,8399
phs000287.c2,7644
phs000287.c3,3643
phs000287.c4,6523
phs000422.c1,23
phs000784.c1,18
phs000820.c1,10
phs000920.c2,5
phs000921.c2,16
phs000946.c1,20
phs000951.c1,5
phs000951.c2,5
phs000954.c1,4
phs000956.c2,64
phs000964.c1,15
phs000964.c2,14
phs000964.c3,15
phs000964.c4,14
phs000974.c1,5
phs000974.c2,5
phs000988.c1,26
phs000993.c1,6
phs000993.c2,6
phs000997.c1,23
phs001001.c1,36
phs001001.c2,27
phs001013.c1,35
phs001013.c2,35
phs001024.c1,23
phs001032.c1,29
phs001040.c1,53
phs001062.c1,8
phs001062.c2,6
phs001074.c2,119
phs001143.c1,46
phs001180.c2,28
phs001189.c1,28
phs001207.c1,44
phs001211.c1,8
phs001211.c2,6
phs001215.c1,36
phs001217.c1,19
phs001218.c2,148
phs001237.c1,7
phs001237.c2,7
phs001238.c

In [9]:
my_query = resource.query()
my_query.filter().delete("\\_topmed_consents\\")
my_query.filter().delete("\\_harmonized_consent\\")
dictionary = resource.dictionary()
output = []
batchSize = 100
badConcepts = []
for consentGroup in consentGroups:
    if ".c0" not in consentGroup:
        print(consentGroup)
        my_query.filter().delete("\\_consents\\")
        my_query.filter().add("\\_consents\\", consentGroup)
        patient_count = my_query.getCount()
        print(patient_count)
        output.append(consentGroup + "," + str(patient_count))

Deleted key: \_topmed_consents\
Deleted key: \_harmonized_consent\
phs000007.c1
Deleted key: \_consents\
13133
phs000007.c2
Deleted key: \_consents\
2021
phs000179.c1
Deleted key: \_consents\
10099
phs000179.c2
Deleted key: \_consents\
272
phs000200.c1
Deleted key: \_consents\
117675
phs000200.c2
Deleted key: \_consents\
25538
phs000209.c1
Deleted key: \_consents\
7440
phs000209.c2
Deleted key: \_consents\
856
phs000280.c1
Deleted key: \_consents\
15610
phs000280.c2
Deleted key: \_consents\
68
phs000284.c1
Deleted key: \_consents\
1473
phs000286.c1
Deleted key: \_consents\
883
phs000286.c2
Deleted key: \_consents\
201
phs000286.c3
Deleted key: \_consents\
2289
phs000286.c4
Deleted key: \_consents\
516
phs000287.c1
Deleted key: \_consents\
5382
phs000287.c2
Deleted key: \_consents\
217
phs000287.c3
Deleted key: \_consents\
2
phs000287.c4
Deleted key: \_consents\
8
phs000422.c1
Deleted key: \_consents\
190
phs000784.c1
Deleted key: \_consents\
1675
phs000820.c1
Deleted key: \_consents\
5

In [10]:
for line in output:
    print(line)

phs000007.c1,13133
phs000007.c2,2021
phs000179.c1,10099
phs000179.c2,272
phs000200.c1,117675
phs000200.c2,25538
phs000209.c1,7440
phs000209.c2,856
phs000280.c1,15610
phs000280.c2,68
phs000284.c1,1473
phs000286.c1,883
phs000286.c2,201
phs000286.c3,2289
phs000286.c4,516
phs000287.c1,5382
phs000287.c2,217
phs000287.c3,2
phs000287.c4,8
phs000422.c1,190
phs000784.c1,1675
phs000820.c1,543
phs000920.c2,4941
phs000921.c2,2106
phs000946.c1,80
phs000951.c1,10333
phs000951.c2,290
phs000954.c1,1293
phs000956.c2,1123
phs000964.c1,797
phs000964.c2,180
phs000964.c3,2131
phs000964.c4,488
phs000974.c1,3432
phs000974.c2,722
phs000988.c1,3478
phs000993.c1,697
phs000993.c2,12
phs000997.c1,173
phs001001.c1,933
phs001001.c2,92
phs001013.c1,1179
phs001013.c2,25
phs001024.c1,128
phs001032.c1,1134
phs001040.c1,118
phs001062.c1,908
phs001062.c2,91
phs001074.c2,250
phs001143.c1,1527
phs001180.c2,4458
phs001189.c1,363
phs001207.c1,637
phs001211.c1,13486
phs001211.c2,60
phs001215.c1,2594
phs001217.c1,3142
phs00121